In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from chatbot_test import ChatBotTs, load_api_key
from OktSentiment import Sentiment

app = Flask(__name__)
CORS(app)

# ChatBot 인스턴스 생성
api_key = load_api_key()
if not api_key:
    raise ValueError("API key not found or invalid. Please check your api_key.txt file.")
chatbot = ChatBotTs(api_key, 'C:\\Users\\smhrd\\Documents\\카카오톡 받은 파일\\원인-대처종합.csv')  # CSV 파일 경로를 적절히 수정하세요
#C:\Users\SMHRD\Documents\카카오톡 받은 파일\원인-대처종합.csv //c:/Users/SMHRD/원인-대처종합.csv
# Sentiment 인스턴스 생성
sentiment_analyzer = Sentiment()

@app.route("/chatbot", methods=["POST"])
def chat():
    data = request.json
    print("Received data:", data)
    user_nick = data.get("user_nick")
    user_input = data.get("input")
    
    if user_input:
        try:
            response = chatbot.process_input(user_nick, user_input)
            if response.get("isProblem"):
                # 형태소 분석 및 감정 분석 수행
                sentiment_results = sentiment_analyzer.parse([user_input])
                # 결과가 딕셔너리 형태로 바꾸고 소수점 소수점 둘쨰자리까지
                if 'polarity' in sentiment_results and len(sentiment_results['polarity']) > 0:
                    sentiment_results = sentiment_results['polarity'][0]
                    sentiment_results = {key: round(value * 100, 2) for key, value in sentiment_results.items()}

                response.update({
                    "sentiment": sentiment_results
                })

            return jsonify(response)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return jsonify({"error": f"An error occurred while processing your request: {str(e)}"}), 500
    else:
        return jsonify({"error": "No input provided"}), 400

if __name__ == "__main__":
    app.run(port=7000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [03/Sep/2024 17:39:19] "OPTIONS /chatbot HTTP/1.1" 200 -


Received data: {'user_nick': '12', 'input': '안녕하세요'}
Processing input: user_nick=12, user_input=안녕하세요


c:\Users\smhrd\Desktop\MindRoad\MindRoad\chatbot_test.py:33: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  ck_chain = LLMChain(llm=self.model, prompt=template)
c:\Users\smhrd\Desktop\MindRoad\MindRoad\chatbot_test.py:34: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = ck_chain.run(user_input=user_input).strip()


Is problem flag: False


...

127.0.0.1 - - [03/Sep/2024 17:48:13] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2024 17:49:39] "OPTIONS /chatbot HTTP/1.1" 200 -


Received data: {'user_nick': '12', 'input': '프로젝트 작업 중 A가 B에게 "Flask 돌려봤어?"라고 물었을 때, B는"네 돌렸어요." 라고 말했고, A가 잠시 후 B를 살펴보자 B는 "pie chart라고 한 거 아니었어요?" 라며 자신이 들은 내용을 부정했어.'}
Processing input: user_nick=12, user_input=프로젝트 작업 중 A가 B에게 "Flask 돌려봤어?"라고 물었을 때, B는"네 돌렸어요." 라고 말했고, A가 잠시 후 B를 살펴보자 B는 "pie chart라고 한 거 아니었어요?" 라며 자신이 들은 내용을 부정했어.
Is problem flag: False


127.0.0.1 - - [03/Sep/2024 17:49:52] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2024 17:50:11] "OPTIONS /chatbot HTTP/1.1" 200 -


Received data: {'user_nick': '12', 'input': '그래서 ㅅㅂ 말이 되는지 안되는지 확률이 얼마 정도 되냐고'}
Processing input: user_nick=12, user_input=그래서 ㅅㅂ 말이 되는지 안되는지 확률이 얼마 정도 되냐고
Is problem flag: False


127.0.0.1 - - [03/Sep/2024 17:50:25] "POST /chatbot HTTP/1.1" 200 -
